In [1]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

In [2]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        rclpy.spin(self.node)

In [3]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
pass 

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [35]:
robot.EENames

['controller_box',
 'pedestal_feet',
 'pedestal',
 'screen',
 'head_camera',
 'right_torso_itb',
 'right_arm_itb',
 'right_hand_camera',
 'right_wrist',
 'right_gripper_l_finger_tip',
 'right_gripper_r_finger_tip',
 'right_gripper_tip',
 'camera_link',
 'right_l4_2',
 'right_l2_2',
 'right_l1_2',
 'torso']

In [4]:
# ros_interface.spin_thread()
t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [ ]:
ros_interface.robot.bodyNames[18]

In [5]:
js=ros_interface.latest_joint_states
header=js.header
names=js.name
poss=np.array(js.position)
vels=np.array(js.velocity) 
j_gripper=poss[-2]
gripper_close=np.allclose(0, j_gripper)

gripper_close

False

In [6]:
q=ros_interface.robot.getJoints()
j_gripper=q[-2]
gripper_close=np.allclose(0, j_gripper)

gripper_close

False

In [7]:
cj=ros_interface.robot.getJoints()
cj 

array([ 2.14776710e-01, -2.89072440e-15, -1.01072043e-02, -1.43806732e+00,
        1.43868005e+00, -1.53588951e+00, -1.43656623e+00,  4.78658229e-01,
        2.99999956e-02, -2.99999937e-02])

In [8]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
motion_detected=False
stop_count=0
for i in range(1000000):
    time.sleep(0.1)
    msg=ros_interface.latest_joint_states
    msgs.append(msg)
    poss=np.array(msg.position)
    d=np.linalg.norm(poss-poss_prev)
    if np.allclose(poss_prev, poss):  # no motion
        if motion_detected:
            # print('motion stopped, d=', d)
            stop_count+=1
            if stop_count>1:
                print('stop')
                break
    elif not motion_detected:
        print('motion detected')
        motion_detected=True
        
    poss_prev=poss.copy()


motion detected
stop


In [9]:
len(msgs)

50

In [36]:
def get_ee(jointmsg):
    msg=jointmsg    
    q =  robot.getJoints() 
    for ind, name in enumerate(msg.name):
        if name in jointMap:
            q[jointMap[name]] = msg.position[ind]
    robot.setJoints(q)
    
    # T=robot.getBodyTransform('right_l6');
    T=robot.getBodyTransform('camera_link');
    xyz=T[0:3,3]
    e1 = rotm2euler.rotationMatrixToEulerAngles(T[0:3, 0:3])
    ee6=np.concatenate([xyz, e1])
    return ee6

In [37]:
ee6_0=get_ee(msgs[0])
ee6_0

array([ 0.49847557, -0.12783336,  0.28214653,  0.24355614,  1.50861656,
        0.28054265])

In [38]:
ee6_n=get_ee(msgs[-1])
ee6_n

array([0.49604321, 0.01439874, 0.28228565, 0.2367697 , 1.50788587,
       0.27416482])

In [39]:
get_ee(ros_interface.latest_joint_states)

array([0.50094439, 0.03512883, 0.2820698 , 0.25118496, 1.50916413,
       0.2877427 ])

In [46]:
def goto_xyzrpy(target, delay=0.1):
    ee6=get_ee(ros_interface.latest_joint_states)
    xd=ee6-target

    alpha = .0001
    I = np.eye(q.shape[0])

    while np.linalg.norm(xd)>0.1:
        ee6=get_ee(ros_interface.latest_joint_states)
        xd=target-ee6 

        J = robot.getJacobian('camera_link')
        
        qd = np.linalg.inv(J.T @ J + alpha * I) @ J.T @ xd

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()

        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

In [47]:
goto_xyzrpy(ee6_0)

In [48]:
goto_xyzrpy(ee6_n)

### approach 2, jp jv

In [20]:
tpos=np.array(msgs[0].position)
cpos=np.array(ros_interface.latest_joint_states.position)
dpos=np.linalg.norm(tpos-cpos)
tpos, cpos, dpos

(array([ 2.14776710e-01, -2.11151211e-15, -1.01072043e-02, -1.43806732e+00,
         1.43868005e+00, -1.53588951e+00, -1.43656623e+00,  4.78658229e-01,
         2.99999956e-02, -2.99999937e-02]),
 array([ 5.23831904e-01, -9.28416381e-15,  4.61013094e-02, -1.49702036e+00,
         1.51172018e+00, -1.46873796e+00, -1.47573721e+00,  7.11586714e-01,
         2.99999956e-02, -2.99999993e-02]),
 0.4096147055171422)

In [49]:
def goto_jointpos(target_jointpos, alpha=0.7, tol=0.1,  delay=0.1):
    cpos=np.array(ros_interface.latest_joint_states.position)
    dpos=np.linalg.norm(target_jointpos-cpos)

    while dpos>tol: 
        cpos=np.array(ros_interface.latest_joint_states.position)
        dpos=np.linalg.norm(target_jointpos-cpos)
 
        qd =(target_jointpos-cpos)*alpha

        cmd_msg = JointCommand()
        cmd_msg.names = robot.jointNames
        cmd_msg.mode = cmd_msg.VELOCITY_MODE
        cmd_msg.header.stamp = rclpy.clock.Clock().now().to_msg()
        cmd_msg.velocity = list(qd)
        cmd_pub.publish(cmd_msg)
        time.sleep(delay)

In [50]:
tpos=np.array(msgs[0].position)
goto_jointpos(tpos)

In [51]:
tpos=np.array(msgs[-1].position)
goto_jointpos(tpos)

In [ ]:
ros_interface.robot.getJoints()